# LSTMのお勉強

*20170621６　Mitsuhiro Ito*

自分で書いてみよう！  
目標　LSTMを利用して夏目漱石っぽい文章を書いてみよう

* [MXnetのRNNの公式チュートリアル](http://mxnet.io/tutorials/r/charRnnModel.html)
* [TJOさんの記事](http://tjo.hatenablog.com/entry/2016/11/08/190000)
* [aidiaryさんの記事](http://qiita.com/elm200/items/6f84d3a42eebe6c47caa)

# 1. 前準備

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


# 2. 文章ファイルの入手

In [4]:
import codecs
path = "souseki_all_hiragana2.txt"
text = codecs.open(path,'r','utf-8').read()
print('corpus length:', len(text)) 
print(text[0:5])

corpus length: 2207475
さんじやま


---
# 3. 文章ファイルをデータ化

まずはdict化する

In [8]:
chars = sorted(list(set(text))) # set :重複しない順序なしコレクション
char_indices = dict((c, i) for i, c in enumerate(chars))#  文字　：　番号
indices_char = dict((i, c) for i, c in enumerate(chars))#　　番号：　文字
print('nb chars:', len(chars))

print (sorted(char_indices))
print (sorted(indices_char))

nb chars: 86
[u'\n', u'\u3001', u'\u3002', u'\u300c', u'\u300d', u'\u3041', u'\u3042', u'\u3043', u'\u3044', u'\u3045', u'\u3046', u'\u3047', u'\u3048', u'\u3049', u'\u304a', u'\u304b', u'\u304c', u'\u304d', u'\u304e', u'\u304f', u'\u3050', u'\u3051', u'\u3052', u'\u3053', u'\u3054', u'\u3055', u'\u3056', u'\u3057', u'\u3058', u'\u3059', u'\u305a', u'\u305b', u'\u305c', u'\u305d', u'\u305e', u'\u305f', u'\u3060', u'\u3061', u'\u3062', u'\u3063', u'\u3064', u'\u3065', u'\u3066', u'\u3067', u'\u3068', u'\u3069', u'\u306a', u'\u306b', u'\u306c', u'\u306d', u'\u306e', u'\u306f', u'\u3070', u'\u3071', u'\u3072', u'\u3073', u'\u3074', u'\u3075', u'\u3076', u'\u3077', u'\u3078', u'\u3079', u'\u307a', u'\u307b', u'\u307c', u'\u307d', u'\u307e', u'\u307f', u'\u3080', u'\u3081', u'\u3082', u'\u3083', u'\u3084', u'\u3085', u'\u3086', u'\u3087', u'\u3088', u'\u3089', u'\u308a', u'\u308b', u'\u308c', u'\u308d', u'\u308f', u'\u3090', u'\u3092', u'\u3093']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1


 ---
 
 次に学習用に４０個の連続した文字のかたまりとその次の文字のデータを抽出する

In [9]:
maxlen = 40 
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))
print('nb next_chars:', len(next_chars))

nb sequences: 735812
nb next_chars: 735812


In [12]:
print(next_chars[0:1])

[u'\u3064']


---
```$ print(tuple(text[0:41]))```　最初の４１文字から...  
(u'\u3055', u'\u3093', u'\u3058', u'\u3084', u'\u307e', u'\u307f', u'\u3061', u'\u3092', u'\u306e', u'\u307c', u'\u308a', u'\u306a', u'\u304c', u'\u3089', u'\u3001', u'\u3053', u'\u3046', u'\u304b', u'\u3093', u'\u304c', u'\u3048', u'\u305f', u'\u3002', u'\n', u'\u3055', u'\u3068', u'\u3057', u'\u3061', u'\u306b', u'\u306f', u'\u305f', u'\u3089', u'\u3051', u'\u3070', u'\u304b', u'\u304f', u'\u304b', u'\u3069', u'\u304c', u'\u305f', u'\u3064')


```$　print(sentences[0:1])```   sentenceに最初の４０文字を格納
[u'\u3055\u3093\u3058\u3084\u307e\u307f\u3061\u3092\u306e\u307c\u308a\u306a\u304c\u3089\u3001\u3053\u3046\u304b\u3093\u304c\u3048\u305f\u3002\n\u3055\u3068\u3057\u3061\u306b\u306f\u305f\u3089\u3051\u3070\u304b\u304f\u304b\u3069\u304c\u305f']


```$　print(next_chars[0:1])```   next_charsに次の文字（４１番目の文字）を格納  
[u'\u3064']

文字３個分ずらして格納していく

---

次に機械学習ができるように行列化する

In [13]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
#  sentenceのリストサイズ　×　 1sentenceの文字数　×　　文字の種類　　のゼロ行列 
# 0:False 1: True
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)# sentenceのリストサイズ　　×　　文字の種類 のゼロ行列　  
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [14]:
X.shape, y.shape

((735812, 40, 86), (735812, 86))

In [18]:
print (sorted(char_indices))

[u'\n', u'\u3001', u'\u3002', u'\u300c', u'\u300d', u'\u3041', u'\u3042', u'\u3043', u'\u3044', u'\u3045', u'\u3046', u'\u3047', u'\u3048', u'\u3049', u'\u304a', u'\u304b', u'\u304c', u'\u304d', u'\u304e', u'\u304f', u'\u3050', u'\u3051', u'\u3052', u'\u3053', u'\u3054', u'\u3055', u'\u3056', u'\u3057', u'\u3058', u'\u3059', u'\u305a', u'\u305b', u'\u305c', u'\u305d', u'\u305e', u'\u305f', u'\u3060', u'\u3061', u'\u3062', u'\u3063', u'\u3064', u'\u3065', u'\u3066', u'\u3067', u'\u3068', u'\u3069', u'\u306a', u'\u306b', u'\u306c', u'\u306d', u'\u306e', u'\u306f', u'\u3070', u'\u3071', u'\u3072', u'\u3073', u'\u3074', u'\u3075', u'\u3076', u'\u3077', u'\u3078', u'\u3079', u'\u307a', u'\u307b', u'\u307c', u'\u307d', u'\u307e', u'\u307f', u'\u3080', u'\u3081', u'\u3082', u'\u3083', u'\u3084', u'\u3085', u'\u3086', u'\u3087', u'\u3088', u'\u3089', u'\u308a', u'\u308b', u'\u308c', u'\u308d', u'\u308f', u'\u3090', u'\u3092', u'\u3093']


---
Xについて 

```print (sorted(char_indices))```   
[u'\n', u'\u3001', u'\u3002', u'\u300c', u'\u300d', u'\u3041', u'\u3042', u'\u3043', u'\u3044', u'\u3045', u'\u3046', u'\u3047', u'\u3048', u'\u3049', u'\u304a', u'\u304b', u'\u304c', u'\u304d', u'\u304e', u'\u304f', u'\u3050', u'\u3051', u'\u3052', u'\u3053', u'\u3054', u'\u3055', u'\u3056', u'\u3057', u'\u3058', u'\u3059', u'\u305a', u'\u305b', u'\u305c', u'\u305d', u'\u305e', u'\u305f', u'\u3060', u'\u3061', u'\u3062', u'\u3063', u'\u3064', u'\u3065', u'\u3066', u'\u3067', u'\u3068', u'\u3069', u'\u306a', u'\u306b', u'\u306c', u'\u306d', u'\u306e', u'\u306f', u'\u3070', u'\u3071', u'\u3072', u'\u3073', u'\u3074', u'\u3075', u'\u3076', u'\u3077', u'\u3078', u'\u3079', u'\u307a', u'\u307b', u'\u307c', u'\u307d', u'\u307e', u'\u307f', u'\u3080', u'\u3081', u'\u3082', u'\u3083', u'\u3084', u'\u3085', u'\u3086', u'\u3087', u'\u3088', u'\u3089', u'\u308a', u'\u308b', u'\u308c', u'\u308d', u'\u308f', u'\u3090', u'\u3092', u'\u3093']

```$　print(sentences[0:2])``` 
[u'\u3055\u3093\u3058\u3084\u307e\u307f\u3061\u3092\u306e\u307c\u308a\u306a\u304c\u3089\u3001\u3053\u3046\u304b\u3093\u304c\u3048\u305f\u3002\n\u3055\u3068\u3057\u3061\u306b\u306f\u305f\u3089\u3051\u3070\u304b\u304f\u304b\u3069\u304c\u305f', u'\u3084\u307e\u307f\u3061\u3092\u306e\u307c\u308a\u306a\u304c\u3089\u3001\u3053\u3046\u304b\u3093\u304c\u3048\u305f\u3002\n\u3055\u3068\u3057\u3061\u306b\u306f\u305f\u3089\u3051\u3070\u304b\u304f\u304b\u3069\u304c\u305f\u3064\u3002\u3058']

上の値からわかるように  
i =0、t=0、char=sentence[0]の0番目 \u3055      char_indices[\u3055]=26   
i =0、t=1、char=sentence[0]の1番目 \u3093      char_indices[\u3093]=85   
i =0、t=2、char=sentence[0]の2番目 \u3058      char_indices[\u3058]=29   
...   
i =0、t=39、char=sentence[0]の39番目 \\u305f   char_indices[\u305f]=36
       
i =1、t=0、char=sentence[1]の0番目 \u3084    char_indices[\u3084]=73  
i =1、t=1、char=sentence[1]の1番目 \u307e   char_indices[\u307e]=77   
i =1、t=2、char=sentence[1]の2番目 \u307f   char_indices[\u307f]=78   
...   
i =1、t=39、char=sentence[1]の39番目 \u3058   char_indices[\u3058]=29

よって　　X =   
[[0,0,0,.. 0,1,0..,0],[0,0,0,....,1],[0,0,0,.. 0,1,0..,0],....[0,0,0,......,1],   
[0,0,0,.. 0,1,0..,0],[0,0,0,.. 0,1,0..,0],[0,0,0,.. 0,1,0..,0],....[0,0,0,.. 0,1,0..,0],..   
.....]]
              

つまり４０文字の集合がonehot表現で735812個集まったもの
          


yについて 

y = [[0,0,0,1,0,0,.......,0],[0,1,0....,0],....]  

つまり次の文字の集合がonehot表現で735812個集まったもの
          

 
 
 ---

 # ４. LSTMを用いた学習と予測

In [144]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))#１２８：ユニット数
model.add(Dense(len(chars)))#ユニット数は文字の種類だけ
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

 LSTM(self, units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, **kwargs)

In [145]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 128)               110080    
_________________________________________________________________
dense_13 (Dense)             (None, 86)                11094     
_________________________________________________________________
activation_13 (Activation)   (None, 86)                0         
Total params: 121,174.0
Trainable params: 121,174.0
Non-trainable params: 0.0
_________________________________________________________________


In [146]:
print(128*65*12) # ユニット数　×　文字の種類　×　（出力、入力、忘却ゲートそれぞれ4個ずつ）？

99840


In [147]:
import sys
from utils import save_model_viz, save_weights, save_hist, plot_hist

In [148]:
RUN_ID = 'LSTM'
save_model_viz(RUN_ID, model)

In [149]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')#予測した値を行列化
    preds = np.log(preds) / temperature#log表示に
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)#多項分布で確率表現
    return np.argmax(probas)

In [150]:
for iteration in range(1, 15):
    hist = model.fit(X, y,batch_size=1024,epochs=1) 
    start_index = random.randint(0, len(text) - maxlen - 1)# ランダムにスタートのインデックスを決める
    sys.stdout.write( '\n iteration = '+ str(iteration) + ' \n')
    sys.stdout.write( '\n selected = ['+initial_40 + '] \n')
    sys.stdout.flush()  
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        generated = ''
        initial_40 = text[start_index: start_index + maxlen]
        sentence = text[start_index: start_index + maxlen]#ランダムに決めたスタートから４０文字を選び
        generated += sentence
        sys.stdout.write( '\n (diversity ='+ str(diversity) + ')\n')
        
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))# １×40×66のゼロ行列を作成
            for t, char in enumerate(sentence):#上で作ったsentenceを予測に使うように行列化
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]#入力x（sentenceの行列表現）に対して次の文字を予測
            next_index = sample(preds, diversity)#予測した文字をindex表現に
            next_char = indices_char[next_index]# index表示（onehot）を文字表示に

            sentence = sentence[1:] + next_char#

            sys.stdout.write(next_char)
            sys.stdout.flush()




Epoch 1/1
735812/735812 [==============================] - 73s - loss: 3.1233      

---
Memo


In [ ]:
from keras.utils.data_utils import get_file

In [ ]:
get_file( '/Users/mitsuhiro/Work/Study/souseki_all','https://github.com/elm200/ltsm_soseki/blob/master/souseki_all_hiragana2.txt')